In [6]:
# =====================================================================
# Excel to CSV Converter for Exercise Database (Normalized Schema v3.0)
# =====================================================================
# This script converts the Excel file into CSV files that match the 
# normalized PostgreSQL schema with separate tables for exercises,
# positions, muscles, progressions, safety constraints, and sports.
# 
# Output files:
# - exercises.csv (main exercise table)
# - exercise_positions.csv (positions junction table)
# - exercise_muscles.csv (muscle recruitment details)
# - exercise_progressions.csv (progression/regression relationships)
# - exercise_safety_constraints.csv (safety flags)
# - exercise_sports.csv (sport similarity)
# - exercises_formatted.json (for LLM/API responses)
# =====================================================================

import pandas as pd
import json
import numpy as np

# Read the Excel file
df = pd.read_excel('bin/OA_Knee_Exercise_Database_old_2.xlsx', sheet_name=0)

# Remove rows with empty exercise names
df = df.dropna(subset=['exercise_name'])

print(f"Processing {len(df)} exercises...")
print("=" * 70)

# Difficulty category mapping
difficulty_mapping = {
    'B': 'beginner',
    'BI': 'beginner_to_intermediate', 
    'I': 'intermediate',
    'IA': 'intermediate_to_advance',
    'A': 'advance'
}

# Position columns mapping
position_columns = {
    'position_SL_stand': 'SL_stand',
    'position_split_stand': 'split_stand', 
    'position_DL_stand': 'DL_stand',
    'position_quadruped': 'quadruped',
    'position_supine_lying': 'supine_lying',
    'position_side_lying': 'side_lying'
}

# Muscle columns mapping
muscle_columns = {
    'muscle_quad': 'quad',
    'muscle_hamstring': 'hamstring',
    'muscle_glute_max': 'glute_max', 
    'muscle_hip_flexors': 'hip_flexors',
    'muscle_glute_med_min': 'glute_med_min',
    'muscle_adductors': 'adductors'
}

# =====================================================================
# Prepare CSV data structures
# =====================================================================
exercises_data = []
positions_data = []
muscles_data = []
progressions_data = []
safety_data = []
sports_data = []
exercises_json = []

exercise_id = 1

for idx, row in df.iterrows():
    try:
        exercise_name = row['exercise_name']
        exercise_name_ch = row.get('exercise_name_ch', '')
        
        # -----------------------------------------------------------------
        # 1. MAIN EXERCISES TABLE
        # -----------------------------------------------------------------
        difficulty_level = int(row['difficulty_level']) if pd.notna(row['difficulty_level']) else 1
        difficulty_cat = row['difficulty_category'] if pd.notna(row['difficulty_category']) else 'B'
        
        exercises_data.append({
            'id': exercise_id,
            'exercise_name': exercise_name,
            'exercise_name_ch': exercise_name_ch if pd.notna(exercise_name_ch) else '',
            'difficulty_level': difficulty_level,
            'difficulty_category': difficulty_mapping.get(difficulty_cat, 'beginner'),
            'core_ipsi': pd.notna(row.get('core_ipsi')) and row.get('core_ipsi') == 'O',
            'core_contra': pd.notna(row.get('core_contra')) and row.get('core_contra') == 'O',
            'toe_touch': pd.notna(row.get('Toe_touch')) and row.get('Toe_touch') == 'O',
            'clinical_summary': ''
        })
        
        # -----------------------------------------------------------------
        # 2. POSITIONS TABLE
        # -----------------------------------------------------------------
        positions_list = []
        for col, pos_name in position_columns.items():
            if pd.notna(row[col]) and row[col] == 'O':
                positions_data.append({
                    'exercise_id': exercise_id,
                    'position': pos_name
                })
                positions_list.append(pos_name)
        
        # -----------------------------------------------------------------
        # 3. MUSCLES TABLE
        # -----------------------------------------------------------------
        muscles_dict = {
            "primary_movers": [],
            "secondary_movers": [],
            "stabiliser": []
        }
        
        for col, muscle_name in muscle_columns.items():
            muscle_type = row[col]
            value_col = col + '_value'
            muscle_value = row[value_col] if pd.notna(row[value_col]) else 0
            
            if pd.notna(muscle_type) and muscle_value > 0:
                muscle_value_int = int(muscle_value)
                
                muscles_data.append({
                    'exercise_id': exercise_id,
                    'muscle': muscle_name,
                    'muscle_type': muscle_type,
                    'muscle_value': muscle_value_int
                })
                
                # Also build for JSON
                muscle_entry = {"muscle": muscle_name, "value": muscle_value_int}
                if muscle_type == 'P':
                    muscles_dict["primary_movers"].append(muscle_entry)
                elif muscle_type == 'N':
                    muscles_dict["secondary_movers"].append(muscle_entry)
                elif muscle_type == 'S':
                    muscles_dict["stabiliser"].append(muscle_entry)
        
        # -----------------------------------------------------------------
        # 4. PROGRESSIONS TABLE
        # -----------------------------------------------------------------
        progression_to_list = []
        progression = row.get('Progression', '')
        if pd.notna(progression):
            prog_list = [prog.strip() for prog in str(progression).split(',') if prog.strip()]
            for prog in prog_list:
                progressions_data.append({
                    'exercise_id': exercise_id,
                    'related_exercise_name': prog,
                    'progression_type': 'progression'
                })
                progression_to_list.append(prog)
        
        progression_from_list = []
        regression = row.get('Regression', '')
        if pd.notna(regression):
            reg_list = [reg.strip() for reg in str(regression).split(',') if reg.strip()]
            for reg in reg_list:
                progressions_data.append({
                    'exercise_id': exercise_id,
                    'related_exercise_name': reg,
                    'progression_type': 'regression'
                })
                progression_from_list.append(reg)
        
        # -----------------------------------------------------------------
        # 5. SAFETY CONSTRAINTS TABLE
        # -----------------------------------------------------------------
        safety_list = []
        safety_flags = row.get('Safety flags', '')
        if pd.notna(safety_flags):
            constraints = [flag.strip() for flag in str(safety_flags).split(',') if flag.strip()]
            for constraint in constraints:
                safety_data.append({
                    'exercise_id': exercise_id,
                    'constraint_type': constraint
                })
                safety_list.append(constraint)
        
        # -----------------------------------------------------------------
        # 6. SPORTS TABLE
        # -----------------------------------------------------------------
        sports_list = []
        activities = row.get('Activities', '')
        if pd.notna(activities):
            activity_list = [sport.strip() for sport in str(activities).split(',') if sport.strip()]
            for sport in activity_list:
                sports_data.append({
                    'exercise_id': exercise_id,
                    'sport': sport
                })
                sports_list.append(sport)
        
        # -----------------------------------------------------------------
        # 7. JSON FORMAT (for API responses)
        # -----------------------------------------------------------------
        exercise_json = {
            "exercise_name": exercise_name,
            "exercise_name_ch": exercise_name_ch if pd.notna(exercise_name_ch) else '',
            "positions": positions_list,
            "muscles": muscles_dict,
            "difficulty": {
                "level": difficulty_level,
                "category": difficulty_mapping.get(difficulty_cat, 'beginner')
            },
            "safety_constraints": safety_list,
            "sport_similarity": sports_list,
            "progression_from": progression_from_list,
            "progression_to": progression_to_list,
            "core_ipsi": pd.notna(row.get('core_ipsi')) and row.get('core_ipsi') == 'O',
            "core_contra": pd.notna(row.get('core_contra')) and row.get('core_contra') == 'O',
            "toe_touch": pd.notna(row.get('Toe_touch')) and row.get('Toe_touch') == 'O',
            "clinical_summary": ""
        }
        exercises_json.append(exercise_json)
        
        print(f"✓ Processed: {exercise_name}")
        exercise_id += 1
        
    except Exception as e:
        print(f"✗ Error processing row {idx}: {e}")

# =====================================================================
# Save all CSV files
# =====================================================================
print("\n" + "=" * 70)
print("Saving CSV files for database import...")
print("=" * 70)

# 1. Main exercises table
df_exercises = pd.DataFrame(exercises_data)
df_exercises.to_csv('exercises.csv', index=False, encoding='utf-8-sig')
print(f"✓ exercises.csv ({len(exercises_data)} rows)")

# 2. Positions
df_positions = pd.DataFrame(positions_data)
df_positions.to_csv('exercise_positions.csv', index=False, encoding='utf-8-sig')
print(f"✓ exercise_positions.csv ({len(positions_data)} rows)")

# 3. Muscles
df_muscles = pd.DataFrame(muscles_data)
df_muscles.to_csv('exercise_muscles.csv', index=False, encoding='utf-8-sig')
print(f"✓ exercise_muscles.csv ({len(muscles_data)} rows)")

# 4. Progressions
df_progressions = pd.DataFrame(progressions_data)
df_progressions.to_csv('exercise_progressions.csv', index=False, encoding='utf-8-sig')
print(f"✓ exercise_progressions.csv ({len(progressions_data)} rows)")

# 5. Safety constraints
df_safety = pd.DataFrame(safety_data)
df_safety.to_csv('exercise_safety_constraints.csv', index=False, encoding='utf-8-sig')
print(f"✓ exercise_safety_constraints.csv ({len(safety_data)} rows)")

# 6. Sports
df_sports = pd.DataFrame(sports_data)
df_sports.to_csv('exercise_sports.csv', index=False, encoding='utf-8-sig')
print(f"✓ exercise_sports.csv ({len(sports_data)} rows)")

# 7. JSON format
with open('exercises_formatted.json', 'w', encoding='utf-8') as f:
    json.dump(exercises_json, f, indent=2, ensure_ascii=False)
print(f"✓ exercises_formatted.json ({len(exercises_json)} exercises)")

# =====================================================================
# Summary
# =====================================================================
print("\n" + "=" * 70)
print("SUMMARY")
print("=" * 70)
print(f"Total exercises processed: {len(exercises_data)}")
print(f"Total positions: {len(positions_data)}")
print(f"Total muscle entries: {len(muscles_data)}")
print(f"Total progression relationships: {len(progressions_data)}")
print(f"Total safety constraints: {len(safety_data)}")
print(f"Total sport associations: {len(sports_data)}")
print("\n✅ All files generated successfully!")
print("\nNext steps:")
print("1. Review the CSV files to ensure data integrity")
print("2. Import into PostgreSQL/Supabase using COPY or the Supabase UI")
print("3. Use exercises_formatted.json for API/LLM responses")
print("=" * 70)

Processing 32 exercises...
✓ Processed: Straight leg raise
✓ Processed: Double leg glute bridging
✓ Processed: Double leg hamstrings bridging
✓ Processed: Single leg glute bridging
✓ Processed: Single leg hamstrings bridging
✓ Processed: Scissors
✓ Processed: 3-months supine hip lowering
✓ Processed: Deadbug
✓ Processed: Side lying Clamshell
✓ Processed: Side plank on knees hip dip
✓ Processed: Side plank on knees hold
✓ Processed: Side plank Clamshell
✓ Processed: Side plank hip abduction
✓ Processed: Copenhagen adductor lv1
✓ Processed: Copenhagen adductor lv2
✓ Processed: Quadruped single limb movement
✓ Processed: Quadruped donkey kick
✓ Processed: Quadruped leg extension
✓ Processed: Quadruped hip abduction
✓ Processed: Birddog
✓ Processed: DL squat
✓ Processed: DL squat with band
✓ Processed: DL squat with adductor squeeze
✓ Processed: Hip hinge
✓ Processed: Split leg squat
✓ Processed: Backward Lunge
✓ Processed: Step up
✓ Processed: Side squat
✓ Processed: Hip hikes
✓ Processed